# IBM Applied Data Science Capstone

**_Opening a New Shopping Mall in Paris, France_**

- Build a dataframe of arrondissement (neighborhoods) in Kuala Paris by web scraping the data from Wikipedia page
- Get the geographical coordinates of the neighborhoods
- Obtain the venue data for the neighborhoods from Foursquare API
- Explore and cluster the neighborhoods
- Select the best cluster to open a new shopping mall
***
### 1. Importing libraries

In [3]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
!pip install geocoder
import geocoder # to get coordinates

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium
import folium # map rendering library

print("Libraries imported.")



     |████████████████████████████████| 102kB 6.8MB/s ta 0:00:011
Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2020.4.5.1 |       hecc5488_0         146 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    branca-0.4.1               |             py_0          26 KB  conda-forge
    certifi-2020.4.5.1         |   py36h9f0ad1d_0         151 KB  conda-forge
    openssl-1.1.1g             |       h516909a_0         2.1 MB  conda-forge
    folium-0.11.0              |             py_0          61 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    br

### 2. Scrap data from Wikipedia page into a DataFrame

In [6]:
# This is link to the Wiki Page for 
url = requests.get("https://en.wikipedia.org/wiki/Arrondissements_of_Paris").text

In [7]:
# parsing the Table containing Arrondissements of Paris with very simple Pandas call

table = pd.read_html('https://en.wikipedia.org/wiki/Arrondissements_of_Paris')[2]
table['Name']
    

0                  Louvre
1                  Bourse
2                  Temple
3          Hôtel-de-Ville
4                Panthéon
5              Luxembourg
6          Palais-Bourbon
7                  Élysée
8                   Opéra
9                Entrepôt
10             Popincourt
11                Reuilly
12               Gobelins
13           Observatoire
14              Vaugirard
15                  Passy
16    Batignolles-Monceau
17       Butte-Montmartre
18        Buttes-Chaumont
19           Ménilmontant
Name: Name, dtype: object

In [43]:
# create a new DataFrame from the list
kl_df = table

#kl_df.drop(kl_df.columns[[0, 2, 3, 4, 5, 6, 7]], axis = 1, inplace = True) 
#kl_df = kl_df.rename(columns={'Name': 'Neighborhood'}, inplace=True)

kl_df['Neighborhood'] =  kl_df['Neighborhood'].astype(str) + ',Paris' 
kl_df.head(20)


,Neighborhood,Latitude,Longitude
0,"Louvre,Paris",48.861020,2.335870
1,"Bourse,Paris",48.869100,2.340220
2,"Temple,Paris",48.864660,2.364260
3,"Hôtel-de-Ville,Paris",48.857095,2.353075
4,"Panthéon,Paris",48.846700,2.344970
5,"Luxembourg,Paris",48.847580,2.340940
6,"Palais-Bourbon,Paris",48.860790,2.318580
7,"Élysée,Paris",48.869313,2.316879
8,"Opéra,Paris",48.882120,2.328060
9,"Entrepôt,Paris",48.842270,2.376120


In [44]:
# print the number of rows of the dataframe
kl_df.shape

# Total 20 Neighborhoods

(20, 3)

### 3. Get the geographical coordinates

In [45]:
# define a function to get coordinates
def get_latlng(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Paris, France'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [46]:
# call the function to get the coordinates, store in a new list using list comprehension
coords = [ get_latlng(neighborhood) for neighborhood in kl_df["Neighborhood"].tolist() ]

In [47]:
coords

[[48.86102000000005, 2.3358700000000567],
 [48.86910000000006, 2.340220000000045],
 [48.86466000000007, 2.3642600000000584],
 [48.8570946465905, 2.3530753470059596],
 [48.84669992224053, 2.344970061168472],
 [48.84758000000005, 2.340940000000046],
 [48.86079001832194, 2.3185800255352524],
 [48.86931307878868, 2.31687879014301],
 [48.88211997446484, 2.3280599888915554],
 [48.842269945034474, 2.3761199194992475],
 [48.85934002553529, 2.375960043857151],
 [48.84498373420427, 2.389387332244692],
 [48.8344775755156, 2.353492216614434],
 [48.83596997446483, 2.3342200183218913],
 [48.83938001110856, 2.300679930607728],
 [48.85847006939234, 2.284189948929627],
 [48.883500000000026, 2.321900000000028],
 [48.89250000000004, 2.344400000000064],
 [48.87937000000005, 2.3851100000000542],
 [48.86314862455058, 2.387301402267264]]

In [54]:
# create temporary dataframe to populate the coordinates into Latitude and Longitude
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])
#df_coords

In [55]:
# merge the coordinates into the original dataframe
kl_df['Latitude'] = df_coords['Latitude']
kl_df['Longitude'] = df_coords['Longitude']

In [56]:
# check the neighborhoods and the coordinates
print(kl_df.shape)
kl_df

(20, 3)


,Neighborhood,Latitude,Longitude
0,"Louvre,Paris",48.861020,2.335870
1,"Bourse,Paris",48.869100,2.340220
2,"Temple,Paris",48.864660,2.364260
3,"Hôtel-de-Ville,Paris",48.857095,2.353075
4,"Panthéon,Paris",48.846700,2.344970
5,"Luxembourg,Paris",48.847580,2.340940
6,"Palais-Bourbon,Paris",48.860790,2.318580
7,"Élysée,Paris",48.869313,2.316879
8,"Opéra,Paris",48.882120,2.328060
9,"Entrepôt,Paris",48.842270,2.376120


In [51]:
# save the DataFrame as CSV file
kl_df.to_csv("kl_df.csv", index=False)

### 4. Create a map of Paris with neighborhoods superimposed on top

In [52]:
# get the coordinates of Kuala Lumpur
address = 'Paris, France'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Paris, France {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Paris, France 48.8566969, 2.3514616.


In [57]:
# create map of Paris using latitude and longitude values
map_kl = folium.Map(location=[latitude, longitude], zoom_start=13)

# add markers to map
for lat, lng, neighborhood in zip(kl_df['Latitude'], kl_df['Longitude'], kl_df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_kl)  
    
map_kl

In [58]:
# save the map as HTML file
map_kl.save('map_kl.html')

### 5. Use the Foursquare API to explore the neighborhoods

In [59]:
# define Foursquare Credentials and Version
CLIENT_ID = 'GFKXYJ13THDHIA35MA5WZMA0JC3KAYRREJ4SDSHSAOUIB25R' # your Foursquare ID
CLIENT_SECRET = 'GL4GXBY1UC0PUKAZLM4N04TPJZLDXALMARXDAGVKSQ4UEEUH' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: GFKXYJ13THDHIA35MA5WZMA0JC3KAYRREJ4SDSHSAOUIB25R
CLIENT_SECRET:GL4GXBY1UC0PUKAZLM4N04TPJZLDXALMARXDAGVKSQ4UEEUH


**Now, let's get the top 100 venues that are within a radius of 2000 meters.**

In [60]:
radius = 2000
LIMIT = 100

venues = []

for lat, long, neighborhood in zip(kl_df['Latitude'], kl_df['Longitude'], kl_df['Neighborhood']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [62]:
# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(2000, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,"Louvre,Paris",48.86102,2.33587,Cour Carrée du Louvre,48.860360,2.338543,Pedestrian Plaza
1,"Louvre,Paris",48.86102,2.33587,Musée du Louvre,48.860847,2.336440,Art Museum
2,"Louvre,Paris",48.86102,2.33587,La Vénus de Milo (Vénus de Milo),48.859943,2.337234,Exhibit
3,"Louvre,Paris",48.86102,2.33587,Vestige de la Forteresse du Louvre,48.861577,2.333508,Historic Site
4,"Louvre,Paris",48.86102,2.33587,Palais Royal,48.863236,2.337127,Historic Site


**Let's check how many venues were returned for each neighorhood**

In [63]:
venues_df.groupby(["Neighborhood"]).count()

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighborhood,,,,,,
"Batignolles-Monceau,Paris",100,100,100,100,100,100
"Bourse,Paris",100,100,100,100,100,100
"Butte-Montmartre,Paris",100,100,100,100,100,100
"Buttes-Chaumont,Paris",100,100,100,100,100,100
"Entrepôt,Paris",100,100,100,100,100,100
"Gobelins,Paris",100,100,100,100,100,100
"Hôtel-de-Ville,Paris",100,100,100,100,100,100
"Louvre,Paris",100,100,100,100,100,100
"Luxembourg,Paris",100,100,100,100,100,100


**Let's find out how many unique categories can be curated from all the returned venues**

In [64]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 178 uniques categories.


In [65]:
# print out the list of categories
venues_df['VenueCategory'].unique()[:50]

array(['Pedestrian Plaza', 'Art Museum', 'Exhibit', 'Historic Site',
       'Theater', 'Plaza', 'Bridge', 'Italian Restaurant', 'Garden',
       'Cosmetics Shop', 'Church', 'Hotel', 'French Restaurant', 'Museum',
       'Coffee Shop', 'Fountain', 'Art Gallery', 'Udon Restaurant',
       'Wine Bar', 'Cocktail Bar', 'Bookstore', 'Bakery', 'Restaurant',
       'Pastry Shop', 'Spa', 'Sandwich Place', 'Ice Cream Shop',
       'Breton Restaurant', "Men's Store", 'Clothing Store', 'Tea Room',
       'Seafood Restaurant', 'Lebanese Restaurant', 'Japanese Restaurant',
       'Creperie', 'Bistro', 'Boutique', 'Beer Bar', 'Electronics Store',
       'Concert Hall', 'Cheese Shop', 'Pub', 'Miscellaneous Shop',
       'Souvenir Shop', 'Park', 'Pharmacy', 'Opera House', 'Burger Joint',
       'Tailor Shop', 'Indie Movie Theater'], dtype=object)

In [68]:
# check if the results contain "Shopping Mall"
"Shopping Mall" in venues_df['VenueCategory'].unique()

True

### 6. Analyze Each Neighborhood

In [29]:
# one hot encoding
kl_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
kl_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [kl_onehot.columns[-1]] + list(kl_onehot.columns[:-1])
kl_onehot = kl_onehot[fixed_columns]

print(kl_onehot.shape)
kl_onehot.head()

(7080, 313)


,Neighborhoods,Accessories Store,Adult Boutique,African Restaurant,American Restaurant,Aquarium,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Auto Dealership,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Badminton Court,Bagel Shop,Bakery,Bar,Baseball Field,Basketball Court,Beer Bar,Beer Garden,Betting Shop,Bistro,Board Shop,Bookstore,Botanical Garden,Boutique,Bowling Alley,Boxing Gym,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Station,Business Service,Butcher,Café,Camera Store,Campground,Cantonese Restaurant,Capitol Building,Casino,Chettinad Restaurant,Chinese Breakfast Place,Chinese Restaurant,Circus,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Bookstore,College Cafeteria,Comedy Club,Comfort Food Restaurant,Concert Hall,Convenience Store,Convention Center,Cosmetics Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dive Shop,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,English Restaurant,Event Space,Exhibit,Fabric Shop,Farm,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Fish Market,Fishing Store,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Service,Food Stand,Food Truck,Frame Store,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Government Building,Grilled Meat Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hainan Restaurant,Hakka Restaurant,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,High School,Hill,Historic Site,History Museum,Hockey Arena,Hong Kong Restaurant,Hookah Bar,Hostel,Hot Spring,Hotel,Hotel Bar,Hotel Pool,Hotpot Restaurant,Housing Development,Hunan Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Indoor Play Area,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Kushikatsu Restaurant,Lake,Latin American Restaurant,Leather Goods Store,Light Rail Station,Lingerie Store,Lounge,Malay Restaurant,Mamak Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Mosque,Motel,Motorcycle Shop,Mountain,Movie Theater,Multiplex,Museum,Music Store,Music Venue,Night Market,Nightclub,Noodle House,North Indian Restaurant,Office,Optical Shop,Other Great Outdoors,Outdoor Event Space,Outlet Store,Pakistani Restaurant,Palace,Park,Performing Arts Venue,Perfume Shop,Pet Café,Pet Store,Pharmacy,Piano Bar,Pie Shop,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Print Shop,Pub,Public Art,Racetrack,Recording Studio,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,Rock Climbing Spot,Rock Club,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Satay Restaurant,Scandinavian Restaurant,Scenic Lookout,Science Museum,Sculpture Garden,Seafood Restaurant,Shabu-Shabu Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Skate Park,Ski Chalet,Ski Lodge,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soccer Stadium,Soup Place,South American Restaurant,South Indian Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Sri Lankan Restaurant,Stables,Stadium,Stationery Store,Steakhouse,Street Art,Street Food Gathering,Supermarket,Supplement Shop,Surf Spot,Sushi Restaurant,Szechuan Restaurant,Taiwanese Restau

**Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category**

In [30]:
kl_grouped = kl_onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(kl_grouped.shape)
kl_grouped

(71, 313)


,Neighborhoods,Accessories Store,Adult Boutique,African Restaurant,American Restaurant,Aquarium,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Auto Dealership,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Badminton Court,Bagel Shop,Bakery,Bar,Baseball Field,Basketball Court,Beer Bar,Beer Garden,Betting Shop,Bistro,Board Shop,Bookstore,Botanical Garden,Boutique,Bowling Alley,Boxing Gym,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Station,Business Service,Butcher,Café,Camera Store,Campground,Cantonese Restaurant,Capitol Building,Casino,Chettinad Restaurant,Chinese Breakfast Place,Chinese Restaurant,Circus,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Bookstore,College Cafeteria,Comedy Club,Comfort Food Restaurant,Concert Hall,Convenience Store,Convention Center,Cosmetics Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dive Shop,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,English Restaurant,Event Space,Exhibit,Fabric Shop,Farm,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Fish Market,Fishing Store,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Service,Food Stand,Food Truck,Frame Store,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Government Building,Grilled Meat Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hainan Restaurant,Hakka Restaurant,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,High School,Hill,Historic Site,History Museum,Hockey Arena,Hong Kong Restaurant,Hookah Bar,Hostel,Hot Spring,Hotel,Hotel Bar,Hotel Pool,Hotpot Restaurant,Housing Development,Hunan Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Indoor Play Area,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Kushikatsu Restaurant,Lake,Latin American Restaurant,Leather Goods Store,Light Rail Station,Lingerie Store,Lounge,Malay Restaurant,Mamak Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Mosque,Motel,Motorcycle Shop,Mountain,Movie Theater,Multiplex,Museum,Music Store,Music Venue,Night Market,Nightclub,Noodle House,North Indian Restaurant,Office,Optical Shop,Other Great Outdoors,Outdoor Event Space,Outlet Store,Pakistani Restaurant,Palace,Park,Performing Arts Venue,Perfume Shop,Pet Café,Pet Store,Pharmacy,Piano Bar,Pie Shop,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Print Shop,Pub,Public Art,Racetrack,Recording Studio,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,Rock Climbing Spot,Rock Club,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Satay Restaurant,Scandinavian Restaurant,Scenic Lookout,Science Museum,Sculpture Garden,Seafood Restaurant,Shabu-Shabu Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Skate Park,Ski Chalet,Ski Lodge,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soccer Stadium,Soup Place,South American Restaurant,South Indian Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Sri Lankan Restaurant,Stables,Stadium,Stationery Store,Steakhouse,Street Art,Street Food Gathering,Supermarket,Supplement Shop,Surf Spot,Sushi Restaurant,Szechuan Restaurant,Taiwanese Restau

In [31]:
len(kl_grouped[kl_grouped["Shopping Mall"] > 0])

39

**Create a new DataFrame for Shopping Mall data only**

In [32]:
kl_mall = kl_grouped[["Neighborhoods","Shopping Mall"]]

In [33]:
kl_mall.head()

,Neighborhoods,Shopping Mall
0,Alam Damai,0.000000
1,"Ampang, Kuala Lumpur",0.010000
2,Bandar Menjalara,0.010000
3,Bandar Sri Permaisuri,0.000000
4,Bandar Tasik Selatan,0.020202


### 7. Cluster Neighborhoods
Run k-means to cluster the neighborhoods in Kuala Lumpur into 3 clusters.

In [34]:
# set number of clusters
kclusters = 3

kl_clustering = kl_mall.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(kl_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 0, 0, 2, 0, 2, 1, 1, 0, 2], dtype=int32)

In [35]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
kl_merged = kl_mall.copy()

# add clustering labels
kl_merged["Cluster Labels"] = kmeans.labels_

In [36]:
kl_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
kl_merged.head()

,Neighborhood,Shopping Mall,Cluster Labels
0,Alam Damai,0.000000,2
1,"Ampang, Kuala Lumpur",0.010000,0
2,Bandar Menjalara,0.010000,0
3,Bandar Sri Permaisuri,0.000000,2
4,Bandar Tasik Selatan,0.020202,0


In [37]:
# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
kl_merged = kl_merged.join(kl_df.set_index("Neighborhood"), on="Neighborhood")

print(kl_merged.shape)
kl_merged.head() # check the last columns!

(71, 5)


,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
0,Alam Damai,0.000000,2,3.057690,101.743880
1,"Ampang, Kuala Lumpur",0.010000,0,3.148494,101.696729
2,Bandar Menjalara,0.010000,0,3.190350,101.625450
3,Bandar Sri Permaisuri,0.000000,2,3.103910,101.712260
4,Bandar Tasik Selatan,0.020202,0,3.072750,101.714610


In [38]:
# sort the results by Cluster Labels
print(kl_merged.shape)
kl_merged.sort_values(["Cluster Labels"], inplace=True)
kl_merged

(71, 5)


,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
70,Wangsa Maju,0.010000,0,3.203910,101.737190
21,Damansara Town Centre,0.010000,0,3.136442,101.690296
23,Dang Wangi,0.020000,0,3.156685,101.698077
48,"Sentul, Kuala Lumpur",0.010000,0,3.175080,101.693050
25,"Federal Hill, Kuala Lumpur",0.020000,0,3.136370,101.685640
27,"Jalan Cochrane, Kuala Lumpur",0.020000,0,3.132903,101.724679
28,Jalan Duta,0.010000,0,3.180025,101.677834
32,Kampung Datuk Keramat,0.010000,0,3.166400,101.730460
57,Taman Desa,0.010000,0,3.102970,101.684710
69,Titiwangsa,0.010000,0,3.180730,101.703210


**Finally, let's visualize the resulting clusters**

In [39]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(kl_merged['Latitude'], kl_merged['Longitude'], kl_merged['Neighborhood'], kl_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [40]:
# save the map as HTML file
map_clusters.save('map_clusters.html')

### 8. Examine Clusters

#### Cluster 0

In [41]:
kl_merged.loc[kl_merged['Cluster Labels'] == 0]

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
70,Wangsa Maju,0.010000,0,3.203910,101.737190
21,Damansara Town Centre,0.010000,0,3.136442,101.690296
23,Dang Wangi,0.020000,0,3.156685,101.698077
48,"Sentul, Kuala Lumpur",0.010000,0,3.175080,101.693050
25,"Federal Hill, Kuala Lumpur",0.020000,0,3.136370,101.685640
27,"Jalan Cochrane, Kuala Lumpur",0.020000,0,3.132903,101.724679
28,Jalan Duta,0.010000,0,3.180025,101.677834
32,Kampung Datuk Keramat,0.010000,0,3.166400,101.730460
57,Taman Desa,0.010000,0,3.102970,101.684710
69,Titiwangsa,0.010000,0,3.180730,101.703210


#### Cluster 1

In [42]:
kl_merged.loc[kl_merged['Cluster Labels'] == 1]

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
12,Bukit Bintang,0.03,1,3.147770,101.708550
15,Bukit Nanas,0.03,1,3.151142,101.699375
22,"Damansara, Kuala Lumpur",0.03,1,3.141906,101.679678
43,"Pudu, Kuala Lumpur",0.04,1,3.133540,101.713070
41,Mont Kiara,0.03,1,3.165320,101.652430
67,Taman U-Thant,0.05,1,3.157700,101.724520
6,Bangsar,0.05,1,3.129200,101.678440
7,Bangsar Park,0.05,1,3.129200,101.678440
37,Lembah Pantai,0.04,1,3.121202,101.663899
66,Taman Tun Dr Ismail,0.03,1,3.152830,101.622710


#### Cluster 2

In [43]:
kl_merged.loc[kl_merged['Cluster Labels'] == 2]

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
58,Taman Ibukota,0.0,2,3.212160,101.715400
56,Taman Connaught,0.0,2,3.082690,101.736890
63,Taman P. Ramlee,0.0,2,3.193940,101.705730
55,Taman Bukit Maluri,0.0,2,3.200660,101.633370
53,Sri Petaling,0.0,2,3.072600,101.682520
59,Taman Len Seng,0.0,2,3.069080,101.742870
65,Taman Taynton View,0.0,2,3.087070,101.736810
68,Taman Wahyu,0.0,2,3.222400,101.671730
60,Taman Melati,0.0,2,3.223570,101.723990
62,Taman OUG,0.0,2,3.210050,101.634508


#### Observations:
Most of the shopping malls are concentrated in the central area of Kuala Lumpur city, with the highest number in cluster 2 and moderate number in cluster 0. On the other hand, cluster 1 has very low number to totally no shopping mall in the neighborhoods. This represents a great opportunity and high potential areas to open new shopping malls as there is very little to no competition from existing malls. Meanwhile, shopping malls in cluster 2 are likely suffering from intense competition due to oversupply and high concentration of shopping malls. From another perspective, this also shows that the oversupply of shopping malls mostly happened in the central area of the city, with the suburb area still have very few shopping malls. Therefore, this project recommends property developers to capitalize on these findings to open new shopping malls in neighborhoods in cluster 1 with little to no competition. Property developers with unique selling propositions to stand out from the competition can also open new shopping malls in neighborhoods in cluster 0 with moderate competition. Lastly, property developers are advised to avoid neighborhoods in cluster 2 which already have high concentration of shopping malls and suffering from intense competition.